In [2]:
import gym
import numpy as np
import random
import tensorflow as tf
import pandas as pd

In [3]:
env = gym.make('FrozenLake-v0')

In [4]:
tf.reset_default_graph()

## Initial the Drawer Class

In [47]:
class Env(object):
    def __init__(self):
        self.s = 0
        self.a = 0
        self.reset()
    
    def reset(self):
        pass
    
    def step(self,state, action):
        pass
    
s = Env()

## Set parameters

In [5]:
state_space_num = 16
action_space_num = 4
x_space_num = 4

num_episodes = 10

## Set Network

In [49]:
inputs1 = tf.placeholder(shape=[1, state_space_num],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([state_space_num,action_space_num],0,0.01))
Qout = tf.matmul(inputs1,W) # predict q value
predict = tf.argmax(Qout,1) # predict action with highest q value

In [50]:
nextQ = tf.placeholder(shape=[1,action_space_num],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

## Train the model

In [51]:
init = tf.global_variables_initializer()

In [53]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        step_in_task = 0
        env = Env()
        s = int(env.s)
        while step_in_task < 27:
            
            #get predict q value and action a with largest q value
            #np.identity(state_space_num)[s:s+1] is [0,0,0,...,1,0,..0] only the position s equals 1
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(state_space_num)[s:s+1]})
            # get new states and rewards from df
            s1,r,d,_ = env.step(s,a)
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            s = s1
            
            step_in_task += 1

TypeError: 'NoneType' object is not iterable

## DQN(improved based on NN)